In [6]:
import sys
!{sys.executable} -m pip install langchain-community langchain
!{sys.executable} -m pip install unstructured
!{sys.executable} -m pip install langchain-openai
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install -U sentence-transformers
!{sys.executable} -m pip install faiss-cpu
!{sys.executable} -m pip install --upgrade huggingface-hub


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

import pandas as pd

# Load reviews
df = pd.read_csv("tripadvisor_hotel_reviews.csv")

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.create_documents(df["Review"].tolist())



In [26]:
print(len(docs))
print(docs[0])

65399
page_content='nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music'


In [9]:
from sentence_transformers import SentenceTransformer

docs_text = [doc.page_content for doc in docs]

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
# Create embeddings + FAISS index
embeddings = model.encode(docs_text)
print(embeddings.shape)

(65399, 384)


In [10]:
import faiss

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Keep mapping from FAISS id → original text
id_to_text = {i: doc.page_content for i, doc in enumerate(docs)}

print(f"Index created with {index.ntotal} vectors of dimension {dimension}")

Index created with 65399 vectors of dimension 384


In [11]:
def semantic_search(query, k=10):
    query_emb = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_emb, k)
    return [id_to_text[i] for i in indices[0]]


In [12]:
# won't work since you don't have openai api quota anymore, try another method
# 
#  from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from dotenv import load_dotenv
# load_dotenv()
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-mini")  # or gpt-4.1

# prompt = PromptTemplate(
#     input_variables=["reviews", "question"],
#     template="""
# You are analyzing hotel reviews.
# Here are the relevant reviews:
# {reviews}

# Answer this question based on the reviews: {question}
# Summarize key patterns and insights in bullet points.
# """
# )

# chain = LLMChain(llm=llm, prompt=prompt)

# # Example query
# query = "What do most the positive reviews have in common?"
# top_reviews = semantic_search(query, k=15)
# answer = chain.run(reviews="\n".join(top_reviews), question=query)
# print(answer)


In [13]:
#lol won't work again becaue it's too heavy to be run locally on my system

# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline


# generator = pipeline(
#     "text-generation",
#     model="mistralai/Mistral-7B-Instruct-v0.1",
#     max_length=512,
#     temperature=0.7
# )

# llm = HuggingFacePipeline(pipeline=generator)

In [16]:
 from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",  # or "google/flan-t5-base" for even less RAM
    task="text2text-generation",
    model_kwargs={
        "max_length": 256,
        "temperature": 0.7,
        "do_sample": True,
        "top_p": 0.95,
        "repetition_penalty": 1.1
    }
)

Device set to use cpu


In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["reviews", "question"],
    template="""
You are analyzing hotel reviews. 
Summarize the key insights from the following reviews in bullet points only. 
Do not include extra text or repeat the question.

Reviews:
{reviews}

Question:
{question}
"""
)

chain = LLMChain(llm=llm, prompt=prompt)


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def rag_query(query, top_k=15):
    # Step 1: Retrieve top-k relevant reviews
    top_reviews = semantic_search(query, k=top_k)
    
    # Step 2: Chunk retrieved reviews for Flan-T5
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    chunks = splitter.split_text("\n".join(top_reviews))
    
    # Step 3: Summarize each chunk
    summaries = [chain.run(reviews=chunk, question=query) for chunk in chunks]
    
    # Step 4: Combine summaries for final answer
    final_summary = "\n".join(summaries)
    return final_summary


In [24]:
query_pos = "What do most positive reviews have in common?"
query_neg = "What do most negative reviews complain about?"

print("Positive reviews insights:\n", rag_query(query_pos))
print("\nNegative reviews insights:\n", rag_query(query_neg))


Positive reviews insights:
 The staff is friendly and helpful.
heeded negative reviews tendency think negative reviews harsh occasionally discount favor positive ones, really wanted believe resort met exceeded praise read site end left little disappointed.there definite positives definite negatives n't overlooked.first positives grounds hotel spectacular,
There are a lot of positive reviews about this hotel. Most of the positive reviews have to do with the staff. Most of the positive reviews have to do with the location.
Ratings impact hotels
There are a lot of positive reviews for this hotel.
The food is excellent and the staff is helpful.
ian allan delightful joy stay, retrospect ca n't imagine negative reviewers coming did expect shangri-la, certainly did n't pay altamont, did later stay s-l
Not expected better, like thank wrote reviews site especially gave good useful tips negative reviews express, good tips came handy times positive reviews realize people satisfied regardless good